# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

/Users/mb/Documents/algorithmic-trading-python/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [30]:
stocks = pd.read_csv('sp_500_stocks.csv')
stock

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [12]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': '  NREALEGNSL(QSKLATGTMBO E/)SDAAC', 'calculationPrice': 'close', 'open': 134, 'openTime': 1608826405439, 'openSource': 'cffiiaol', 'close': 126.899, 'closeTime': 1627306006092, 'closeSource': 'cofiilaf', 'high': 130.5, 'highTime': 1628713240635, 'highSource': 'd ticrdie5npuy al1eem e', 'low': 132.09, 'lowTime': 1657791854299, 'lowSource': 'n p1tduemrc5e yiaedlei ', 'latestPrice': 131.517, 'latestSource': 'Close', 'latestTime': 'December 18, 2020', 'latestUpdate': 1624104257394, 'latestVolume': 201338085, 'iexRealtimePrice': 131.2, 'iexRealtimeSize': 152, 'iexLastUpdated': 1643308587073, 'delayedPrice': 127.12, 'delayedPriceTime': 1664828694564, 'oddLotDelayedPrice': 129.29, 'oddLotDelayedPriceTime': 1682630337902, 'extendedPrice': 130.4, 'extendedChange': 0.658, 'extendedChangePercent': 0.0051, 'extendedPriceTime': 1642848991368, 'previousClose': 134.5, 'previousVolume': 94404317, 'change': -2.08, 'changePercent': -0.01

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [16]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price, market_cap)

131.517 2169812760453


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [51]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [28]:
final_data_frame.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
       index=my_columns 
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,131.517,2169812760453,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [37]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index=my_columns),
        ignore_index=True
    )


In [38]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,119.800,37673398266,N/A
1,AAL,16.600,10298747799,N/A
2,AAP,161.690,11329333488,N/A
3,AAPL,129.655,2196730661642,N/A
4,ABBV,105.120,186004137728,N/A
...,...,...,...,...
500,YUM,109.830,33504488567,N/A
501,ZBH,156.920,31865490411,N/A
502,ZBRA,391.000,20767749344,N/A
503,ZION,43.110,7058280119,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [39]:
import sqlite3 as db
conn = db.connect('stock_db')
final_dataframe.to_sql(name="stock_1912", con=conn)

/Users/mb/Documents/algorithmic-trading-python/venv/lib/python3.8/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [40]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [58]:
symol_groups =list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symol_groups)):
    symbol_strings.append(','.join(symol_groups[i]))
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                    ],
                    index=my_columns),
                    ignore_index = True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,121.700,37199642381,N/A
1,AAL,17.240,10010070406,N/A
2,AAP,165.310,10882823973,N/A
3,AAPL,132.474,2256359297417,N/A
4,ABBV,105.030,190745280328,N/A
...,...,...,...,...
500,YUM,114.020,34441925603,N/A
501,ZBH,158.780,32902853259,N/A
502,ZBRA,374.000,20051347992,N/A
503,ZION,42.530,6804064942,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [69]:
portfolio_size = input('Enter the value of your portfolio')
try:
    val = float(portfolio_size)
    portfolio_size = val
except ValueError:
    print("That is not number \nPleses enter integer value")
    portfolio_size = input('Enter the value of your portfolio')
    val = float(portfolio_size)
    portfolio_size = val

Enter the value of your portfolio10000000


In [75]:
position_size = portfolio_size/len(final_dataframe.index)
position_size
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe    

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,121.700,37199642381,162
1,AAL,17.240,10010070406,1148
2,AAP,165.310,10882823973,119
3,AAPL,132.474,2256359297417,149
4,ABBV,105.030,190745280328,188
...,...,...,...,...
500,YUM,114.020,34441925603,173
501,ZBH,158.780,32902853259,124
502,ZBRA,374.000,20051347992,52
503,ZION,42.530,6804064942,465


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [109]:
writer = pd.ExcelWriter('recomended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recomended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [110]:
backround_color = '#0a0a3'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': backround_color,
        'border':1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0. 00',
        'font_color': font_color,
        'bg_color': backround_color,
        'border':1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': backround_color,
        'border':1
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [111]:
# writer.sheets['Recomended Trades'].set_column('A:A', 18,  string_format)
# writer.sheets['Recomended Trades'].set_column('B:B', 18,  string_format)
# writer.sheets['Recomended Trades'].set_column('C:C', 18,  string_format)
# writer.sheets['Recomended Trades'].set_column('D:D', 18,  string_format)
# writer.save()

writer.sheets['Recomended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recomended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recomended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recomended Trades'].write('D1', 'Number of Shares to Buy', integer_format)


0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [112]:
column_format = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', dollar_format],
}
for column in column_format.keys():
    writer.sheets['Recomended Trades'].set_column(f'{column}:{column}', 18, column_format[column][1])
    writer.sheets['Recomended Trades'].write(f'{column}1', column_format[column][0], column_format[column][1])
    


## Saving Our Excel Output

Saving our Excel file is very easy:

In [113]:
writer.save()